# Tutorial 1: Names, Types, and Binding in Julia

This first tutorial walks you through the basics of variable names, types, and binding behaviors in Julia without bogging you down in the specifics and jargons of language design. Which is honestly quite a lot to learn already: Julia has a few idiosyncracies that are either unseen in or remixes from popular languages like Python, C++, Lisp, Matlab, and so on. 

## 1. Names

Let's start with variable names. Actually, let's start with the 29 reserved keywords that must *not* be variable names, which are `baremodule`, `begin`, `break`, `catch`, `const`, `continue`, `do`, `else`, `elseif`, `end`, `export`, `false`, `finally`, `for`, `function`, `global`, `if`, `import`, `let`, `local`, `macro`, `module`, `quote`, `return`, `struct`, `true`, `try`, `using`, and `while`. Other than those, you can name your variables in whatever ways you want, as long as they

- do not start with any digit or symbols other than letters (a-z or A-Z), underscore `_`, emojis, and a handful of other symbols whose Unicode points are greater than 00A0 (the best way to find out which ones exactly is, unfortunately, trial-and-error), and
- continue with all such symbols as well as `!` and digits (0-9) (and a few others that are also unfortunately best found out by trial-and-error).

To make things less confusing and more readable, the following conventions created by the Julia community should be adopted:

- Names should all start with Roman letters.
- Variable names should be in `snake_case` (although they are encouraged to be short and not having any underscore "`_`") and all lowercase.
- Constant names should be capitalized and in `SNAKE_CASE`.
- Structs, Types and Modules should begin with a capital letter and be in upper `CamelCase`.

Some Julia programmers use Greek letters and some uncommon symbols to name mathematical variables, which are apparently accepted by the Julia community. However, I find them a bit inconvenient: to type a symbol like Σ, you either need to have a keyboard mapping that supports it or type `\Sigma`*-tab* in the Julia shell.

In [ ]:
# Examples of variable names that are legal but unconventional and/or ugly:
😅yikes😅 = "yikes"
xₐ = "xa"

# Examples of variable and constant names that are legal and conventional:
x = 2
xa = 2
x_other = 2
X_CONSTANT = 2

## 2. Types

Julia is dynamically (and relatively strongly) typed, but it allows specifying types like statistically typed languages abeit in slightly dissimilar manners. For example, the following lines are legal and prompt the Julia compiler to infer the type of each variable:

### 2.1. Dynamic versus static typing

In [87]:
integer_d = 3
unsigned_d = 0x123456789abcdef
float_d = 3.5
char_d = '3'
string_d = "three"
boolean_d = true
array_d = [1, 2, 3]
dictionary_d = Dict("one" => 1, "two" => 2, "three" => 3)

println("Type of `integer_d`: ", typeof(integer_d))
println("Type of `unsigned_d`: ", typeof(unsigned_d))
println("Type of `floating_d`: ", typeof(float_d))
println("Type of `char_d`: ", typeof(char_d))
println("Type of `string_d`: ", typeof(string_d))
println("Type of `boolean_d`: ", typeof(boolean_d))
println("Type of `array_d`: ", typeof(array_d))
println("Type of `dictionary_d`: ", typeof(dictionary_d))

Type of `integer_d`: Int64
Type of `unsigned_d`: UInt64
Type of `floating_d`: Float64
Type of `char_d`: Char
Type of `string_d`: String
Type of `boolean_d`: Bool
Type of `array_d`: Vector{Int64}
Type of `dictionary_d`: Dict{String, Int64}


The following lines are also legal, where the type each variable is explicitly declared:

In [88]:
let 
    integer_s::Int = 3 
    unsigned_s::UInt = 0x123456789abcdef
    float_s::AbstractFloat = 3.5
    char_s::Char = '3'
    string_s::String = "three"
    boolean_s::Bool = true
    array_s::Array = [1, 2, 3]
    dictionary_s::Dict = Dict("one" => 1, "two" => 2, "three" => 3)
    println("Type of `integer_s`: ", typeof(integer_s))
    println("Type of `unsigned_s`: ", typeof(unsigned_s))
    println("Type of `float_s`: ", typeof(float_s))
    println("Type of `char_s`: ", typeof(char_s))
    println("Type of `string_s`: ", typeof(string_s))
    println("Type of `boolean_s`: ", typeof(boolean_s))
    println("Type of `array_s`: ", typeof(array_s))
    println("Type of `dictionary_s`: ", typeof(dictionary_s))
end


Type of `integer_s`: Int64
Type of `unsigned_s`: UInt64
Type of `float_s`: Float64
Type of `char_s`: Char
Type of `string_s`: String
Type of `boolean_s`: Bool
Type of `array_s`: Vector{Int64}
Type of `dictionary_s`: Dict{String, Int64}


However, did you notice anything interesting about the code above? The first thing is that *variable types can only be explicitly declared within local scopes* which can be enclosed by `let` blocks (or other scope blocks such as `while` loops). The Julia REPL recognizes the variable in the following line as global and forbids its type being explicit declared:

In [20]:
integer_s::Int = 3 

LoadError: syntax: type declarations on global variables are not yet supported

The second peculiar thing is that even though the types are declared, *the Julia compiler further infers them as something else*. This is called *type aliasing*; for instance, `Int` is an alias of `Int64` in 64-bit systems; therefore, calling the `typeof` method on an object declared as `Int` actually returns `Int64`. This behavior implies one thing that differentiates Julia from other statically typed languages: type declaration most likely does *not* introduce any performance gain, since the compiler has to infer the variable's type anyway regardless of the declared type. Take a look at the next block of code, which shows that there is almost no difference in execution time between dynamic and static typing and among different types sharing the same supertype: 

In [57]:
import Pkg; Pkg.add("BenchmarkTools")
using BenchmarkTools

function loop_init_float_dynamic()
    for i in 0:10^10 x = 3.5 end
end

function loop_init_abstract_float()
    for i in 0:10^10 x::AbstractFloat = 3.5 end
end

function loop_init_float32()
    for i in 0:10^10 x::Float32 = 3.5 end
end

function loop_init_float64()
    for i in 0:10^10 x::Float64 = 3.5 end
end

@btime loop_init_float_dynamic()
@btime loop_init_abstract_float()
@btime loop_init_float32()
@btime loop_init_float64()

  7.536 ns (0 allocations: 0 bytes)
  7.535 ns (0 allocations: 0 bytes)
  7.539 ns (0 allocations: 0 bytes)
  7.534 ns (0 allocations: 0 bytes)


### 2.2. Primitive types and conversion

Julia has a set of standard primitive types that give rise to all concrete data types, namely `Float16`, `Float32`, `Float64`, `Bool`, `Char`, `Int8`, `UInt8`, `Int16`, `UInt16`,`Int32`, `UInt32`,`Int64`, `UInt64`,`Int128`, and `UInt128`. Julia also has a bunch of built-in abstract types such as `Number`, which relate to one another and to concrete and/or primitive types in a hierarchy whose root is the type `Any`. To have a glimpse of that hierarchy, take a look at the following code which traverses through the supertypes of the objects we constructed a few blocks above:

In [100]:
function typetraverse(x)
    sub = typeof(x)
    super = supertype(sub)
    print(sub)
    while sub != super 
        print(" -> ", super)
        sub = super 
        super = supertype(sub)
    end
end

examples = [integer_d, unsigned_d, float_d, char_d, string_d, boolean_d, 
            array_d, dictionary_d]

for e in examples
    print("Traverse `$e`: \t")
    typetraverse(e)
    println()
end


Traverse `3`: 	Int64 -> Signed -> Integer -> Real -> Number -> Any
Traverse `81985529216486895`: 	UInt64 -> Unsigned -> Integer -> Real -> Number -> Any
Traverse `3.5`: 	Float64 -> AbstractFloat -> Real -> Number -> Any
Traverse `3`: 	Char -> AbstractChar -> Any
Traverse `three`: 	String -> AbstractString -> Any
Traverse `true`: 	Bool -> Integer -> Real -> Number -> Any
Traverse `[1, 2, 3]`: 	Vector{Int64} -> DenseVector{Int64} -> AbstractVector{Int64} -> Any
Traverse `Dict("two" => 2, "one" => 1, "three" => 3)`: 	Dict{String, Int64} -> AbstractDict{String, Int64} -> Any


Looking at the results above might give us some intuition about the fact that in some cases, when the variables are explicitly typed, the value bound to one variable can be converted into something else upon being assigned to another variable, *e.g.,*

In [164]:
let 
    x::Int16 = 2
    y::UInt128 = 3
    println("Type of y is ", typeof(y))
    x = y
    print("Type of x is ", typeof(x))
end

Type of y is UInt128
Type of x is Int16

which also makes the following code legal,

In [167]:
let 
    # implicit conversion
    x::Char = '2'
    y::Int64 = 5
    z = x+y
    println("Type of z is ", typeof(z))
    println("z = ", z)

    # implcit, widening conversion
    a::Float64 = 3.5
    b::Int64 = 2
    c = a + b
    println("Type of c is ", typeof(c))
    println("c = ", c)
end

Type of z is Char
z = 7
Type of c is Float64
c = 5.5


However, conversion is not always possible, *e.g.*,

In [169]:
let
    x::Int64 = 3 
    y::String = "three"
    x = y
    println("Type of `integer_s`: ", typeof(x))
end

LoadError: MethodError: [0mCannot `convert` an object of type [92mString[39m[0m to an object of type [91mInt64[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::Ptr[39m) where T<:Integer at /usr/share/julia/base/pointer.jl:23
[0m  convert(::Type{T}, [91m::T[39m) where T<:Number at /usr/share/julia/base/number.jl:6
[0m  convert(::Type{T}, [91m::Number[39m) where T<:Number at /usr/share/julia/base/number.jl:7
[0m  ...

In [168]:
let
    x::Int64 = 3 
    y::Float64 = 0.5
    x = y
    println("Type of `x`: ", typeof(x))
end

LoadError: InexactError: Int64(0.5)

### 2.3. Mutable versus immutable types

Notice that in all such cases, variables with declared types are *immutable* because they are primitive types. Some built-in composite types (*i.e.,* non-primitive types) in Julia are mutable, *i.e.* a variable's value can be changed after it is constructed. An example is the `Array` type:

In [174]:
let
    arr::Array = [1, 2, 3]
    append!(arr, 4)
    print(arr)
end

[1, 2, 3, 4]

However, composite type objects declared by the `struct` keyword are immutable:

In [175]:
struct MyObject 
    x::Int
    y::Float64
end

m = MyObject(0, 0.5)
m.x = 3

LoadError: setfield!: immutable struct of type MyObject cannot be changed

To make `MyObject` mutable, it must be declared by the `mutable struct` keyword:

In [178]:
mutable struct MyObject2 
    x::Int
    y::Float64
end

m = MyObject2(0, 0.5)
m.x = 3

3